In [1]:
import keras
from keras.layers import Input, Dense, Dot, Softmax
from keras.models import Model

import numpy as np

Using TensorFlow backend.


We will make 6 dimensional clothing vectors, and train the way to find transform them into the 2 dimensional "shoe vector" space

In [2]:
CLOTHING_DIM = 6
SHOE_DIM = 2

clothe_input = Input(shape=[CLOTHING_DIM], name='clothing_input')
clothe_transform = Dense(SHOE_DIM, activation='linear', name='clothe_to_shoe')(clothe_input)

shoe_input = Input(shape=[SHOE_DIM], name='shoe_input')

prod = Dot(axes=1, name='DotProduct')([clothe_transform, shoe_input])
pred = Dense(1, name='Prediction',  activation='sigmoid')(prod)

model = Model(inputs=[clothe_input, shoe_input], outputs=[pred])
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
clothing_input (InputLayer)     (None, 6)            0                                            
__________________________________________________________________________________________________
clothe_to_shoe (Dense)          (None, 2)            14          clothing_input[0][0]             
__________________________________________________________________________________________________
shoe_input (InputLayer)         (None, 2)            0                                            
__________________________________________________________________________________________________
DotProduct (Dot)                (None, 1)            0           clothe_to_shoe[0][0]             
                                                                 shoe_input[0][0]                 
__________

Now let's make up some data to fit (4 examples of clothes are shoe pairings).

In the real example, it would be "clothing_vector shoe_vector" and the corresponding rating. Note the same "clothing vector" and "shoe_vector" can appear on multiple rows.

In [9]:
train_clothe = np.array([[0,0,1,1,-1,-1], [1,0.2,0,0.2,-1,-1], [1,1,1,0.3,-1,0], [1,0.3,0.4,0.2,0.2,-1]]).astype('float')
train_shoe = np.array([[1,1],[0.2,1],[0.3,0.2],[-0.1,0.2]]).astype('float')
ratings = np.array([1,0,1,0])

Fit the model

In [10]:
model.fit([train_clothe, train_shoe], ratings, epochs=8000, verbose=0)

Notice how close these are to the made up ratings

In [12]:
model.predict([train_clothe, train_shoe])

array([[1.0000000e+00],
       [1.1837942e-04],
       [9.9985456e-01],
       [8.0105133e-04]], dtype=float32)

## Now get the transformation from clothing space to shoe space

Let's get just the transformation that takes a dress vector (dim `CLOTHING_DIM`=6) and returns a shoe vector (dim `SHOE_DIM`=2)

In [13]:
dress_to_shoe = Model(inputs=[clothe_input], outputs=[clothe_transform])
shoe_vectors = dress_to_shoe.predict(train_clothe)
shoe_vectors

array([[ 9.540722 , -4.370069 ],
       [10.104689 , -4.1889424],
       [12.708819 , -4.3997965],
       [ 8.85155  , -3.6856136]], dtype=float32)

In [14]:
# This is the dot product, check results from above
np.sum(shoe_vectors * train_shoe, axis=1)

array([ 5.17065287, -2.1680047 ,  2.93268652, -1.62227774])

We can also extract the weights from that layer directly and build the transformation

$$\text{shoe vector} = Ax + b$$

where $x$ is the `dress_vector`

In [15]:
A, b = model.get_layer(name='clothe_to_shoe').get_weights()
A

array([[ 3.8497903 , -1.2229562 ],
       [ 1.5255924 , -0.9807685 ],
       [ 2.1056201 ,  0.2662897 ],
       [ 1.856652  , -2.3331583 ],
       [-1.8732889 ,  0.41240817],
       [-0.9076289 ,  0.5407866 ]], dtype=float32)

Let's manually construct the clothes vectors

In [16]:
output2 = train_clothe @ A + b
output2

array([[ 9.54072177, -4.37006932],
       [10.10468884, -4.18894231],
       [12.70881925, -4.39979661],
       [ 8.85154949, -3.68561347]])

Let's show this is the same (up to rounding) as the `shoe_vectors` we found the hard way earlier

In [17]:
np.allclose(output2, shoe_vectors)

True

In [18]:
# numerically
output2 - shoe_vectors

array([[-1.19209290e-07, -2.98023224e-07],
       [ 1.90734863e-07,  1.19209290e-07],
       [-1.43051148e-07, -1.19209290e-07],
       [-6.07967376e-07,  1.60932541e-07]])

In [19]:
# Hopefully in your real model these numbers will be positive, especially the first one
model.get_layer(name='Prediction').get_weights()

[array([[3.50489]], dtype=float32), array([-1.4428793], dtype=float32)]

## New clothes

The process for new clothes:
1. Get the clothe_vector using the pre-existing model
2. Use `dress_to_shoe.predict` to get the corresponding shoe vector
3. Now look in the neighborhood of those returned shoe vectors. Use a neighborhood model here to find similar shoes